<a href="https://colab.research.google.com/github/Tadashi1999/Repo_tst/blob/master/%E3%83%AD%E3%83%BC%E3%82%BD%E3%82%AF%E8%B6%B3%2B%E3%83%9C%E3%83%AA%E3%83%90%E3%83%B3%EF%BC%8B%E5%87%BA%E6%9D%A5%E9%AB%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google colab,Ta-lib,Plotlyを用いた株価テクニカルチャートの表示
#akatak's blog,「PlotlyとTalibでテクニカルチャート分s系を描く」2019-11-23より、一部変更


In [ ]:
#Talibのインストール
!wget http://prdownloads.sourceforge.net/talib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib.dpkg.in
!/configure --prefix=/usr
!make
!make install
!pip install Ta-lib
import talib as ta

In [ ]:
#株価の取得方法
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
starts=datetime(2019,10,1)
end=datetime(2020,10,28)
df=web.DataReader('2395.jp','stooq',start,end)

In [ ]:
#複数のテクニカル分析
#一般ライブラリのインストール
import talib as ta
import numpy as np
import pandas as pd
!pip install plotly --upgrade

import plotly.offline as pyo
#import chart_studio.plotly as py
import plotly.graph_objects as go

hnp=np.array(df['High']) #npアレイ形式に変換
lnp=np.array(df['Low'])
cnp=np.array(df['Close'])
onp=np.array(df['Open'])
c=df['Close'] #何故かMACDではこれで動作する


#単純移動平均
sma5=ta.SMA(c,timeperiod=5)
sma20=ta.SMA(c,timeperiod=20)
sma50=ta.SMA(c,timeperiod=50)

#Plotlyでは DataFrameを使うので、以下を追加
df['SMA5']=sma5
df['SMA20']=sma20
df['SMA50']=sma50

#MACD
macd,macd_sig,macd_hist=ta.MACD(c,fastperiod=5,slowperiod=10,signalperiod=9)
df["MACD"]=macd
df["MACD_SIG"]=macd_sig
df["MACD_HIST"]=macd_hist

#ボリンジャーバンド
u_band,m_band,l_band=ta.BBANDS(c,timeperiod=5,nbdevup=2,nbdevdn=2,matype=0)
df['BBAND_U']=u_band
df['BBAND_M']=m_band
df['BBAND_L']=l_band

#Plotlyでチャート
layout={
    'hight':1000,
    'title':{'text':'銘柄名','x':0.5},
    'titlefont':{'size':25},
    'xaxis':{'title':"",'rangeslider':{'visible':False}},
    'yaxis':{'domain':[.55,1],'title':"株価（円)",'side':"left",'tickformat':','},
    'yaxis2':{'domain':[.45,.55],'title':"MACD",'side':"right"},
}#akatak'sのまま

#ローソク足、ボリンジャーバンド、移動平均、MACD、出来高
trace1=go.Candlestick(x=df.index,open=onp,high=hnp,low=lnp,close=cnp,yaxis="y2",name="銘柄名")
#trace2=go.Scatter(yaxis="y2",x=df.index,y=df["SMA5"],name='SMA5',line=dict(color='Black',width=0.5))
#trace3=go.Scatter(yaxis="y2",x=df.index,y=df["SMA20"],name='SMA20',line=dict(color='Black',width=0.5))
#trace4=go.Scatter(yaxis="y2",x=df.index,y=df["SMA50"],name='SMA50',line=dict(color='darkblue',width=1))
trace5=go.Scatter(yaxis="y2",x=df.index,y=df["MACD"],name='MACD',line=dict(color='blue',width=1.5))
trace6=go.Scatter(yaxis="y2",x=df.index,y=df["MACD_SIG"],name='MACD_SIG',line=dict(color='red',width=1))
trace7=go.Bar(yaxis="y2",x=df.index,y=np.array(df['Volume'])/1000000,name="出来高")#高さ調整
trace8=go.Scatter(yaxis="y2",x=df.index,y=df["BBAND_U"],name="Bバンド＋2σ",line=dict(color='darkblue',width=1))
trace9=go.Scatter(yaxis="y2",x=df.index,y=df["BBAND_M"],name="Bバンドー",line=dict(color='black',width=0.5))
trace10=go.Scatter(yaxis="y2",x=df.index,y=df['BBAND_L'],name="Bバンド-2σ",line=dict(color='red',width=1))
data=[trace1,trace8,trace9,trace10]
go.Figure(data).show()     #Google ColabではこれでOK

#出来高、MACDのみ表示
fig=go.Figure()
fig.add_trace(go.Bar(yaxis="y2",x=df.index,y=np.array(df['Volume'])/100000,name="出来高"))
fig.add_trace(go.Scatter(yaxis="y2",x=df.index,y=df["MACD"],name="MACD"))
fig.add_trace(go.Scatter(yaxis="y2",x=df.index,y=df["MACD_SIG"],name="MACD_SIG"))

#問題点　営業日のみの表記ができない